In [1]:
# Import libraries here

from glob import glob

import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted


In [2]:
# Build your `wrangle` function
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath, encoding='latin-1')

    # Subset data: Apartments in "Distrito Federal", less than $100,000
    mask_ba = df["place_with_parent_names"].str.contains("Distrito Federal")
    mask_apt = df["property_type"] == "apartment"
    mask_price = df["price_aprox_usd"] < 100_000
    df = df[mask_ba & mask_apt & mask_price]

    # Subset data: Remove outliers for "surface_covered_in_m2"
    low, high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low, high)
    df = df[mask_area]

    # Split "lat-lon" column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)
    
    # Step 4: Create "borough" feature from "place_with_parent_names" column
    df['borough'] = df['place_with_parent_names'].str.split('|').str[1]
    
    #     Drop columns with more than 50% null values
    null_percentage = df.isnull().mean() * 100
    columns_to_drop1 = null_percentage[null_percentage > 50].index
    df.drop(columns=columns_to_drop1, inplace=True)
    

   # Step 6: Drop columns with low/high cardinality, leakage, or multicollnearity issues
    columns_to_drop = ['property_type', 'operation', 'place_with_parent_names', 
                       'currency', 'price', 'price_per_m2', 'price_aprox_local_currency', 
                       'properati_url']
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df


In [3]:
# Use this cell to test your wrangle function and explore the data
frame1 = wrangle("data/mexico-city-real-estate-1.csv")
print(frame1.info())
frame1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1101 entries, 11 to 4605
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price_aprox_usd        1101 non-null   float64
 1   surface_covered_in_m2  1101 non-null   float64
 2   lat                    1041 non-null   float64
 3   lon                    1041 non-null   float64
 4   borough                1101 non-null   object 
dtypes: float64(4), object(1)
memory usage: 51.6+ KB
None


,price_aprox_usd,surface_covered_in_m2,lat,lon,borough
11,94022.66,57.0,23.634501,-102.552788,Benito Juárez
20,70880.12,56.0,19.402413,-99.095391,Iztacalco
21,68228.99,80.0,19.357820,-99.149406,Benito Juárez
22,24235.78,60.0,19.504985,-99.208557,Azcapotzalco
26,94140.20,50.0,19.354219,-99.126244,Coyoacán


In [4]:
files = glob("data/mexico-city-real-estate-*.csv")

files

['data\\mexico-city-real-estate-1.csv',
 'data\\mexico-city-real-estate-2.csv',
 'data\\mexico-city-real-estate-3.csv',
 'data\\mexico-city-real-estate-4.csv',
 'data\\mexico-city-real-estate-5.csv']

In [5]:
frames = [wrangle(file) for file in files]
frames[0].head()



,price_aprox_usd,surface_covered_in_m2,lat,lon,borough
11,94022.66,57.0,23.634501,-102.552788,Benito Juárez
20,70880.12,56.0,19.402413,-99.095391,Iztacalco
21,68228.99,80.0,19.357820,-99.149406,Benito Juárez
22,24235.78,60.0,19.504985,-99.208557,Azcapotzalco
26,94140.20,50.0,19.354219,-99.126244,Coyoacán


In [6]:
df = pd.concat(frames, ignore_index=True)
df.shape

(5473, 5)

In [ ]:
# Build histogram
plt.hist(df['price_aprox_usd'])


# Label axes

plt.xlabel('Price [$]')
plt.xlabel('Count')

# Add title

plt.title('Distribution of Apartment Sizes');




In [ ]:
# Build scatter plot

plt.scatter(x= df['surface_covered_in_m2'], y=df['price_aprox_usd'])



# Label axes
plt.xlabel('Area [sq meters]')
plt.ylabel('Price [USD]')

# Add title
plt.title('Mexico City: Price vs. Area');



In [ ]:
fig = px.scatter_mapbox(
    df,  # Our DataFrame
    lat= 'lat',
    lon= 'lon',
    width=600,  # Width of map
    height=600,  # Height of map
    color= 'price_aprox_usd',
    hover_data=["price_aprox_usd"],  # Display price when hovering mouse over house
)

fig.update_layout(mapbox_style="open-street-map")

fig.show()

In [8]:
# Split data into feature matrix `X_train` and target vector `y_train`.

features = ['surface_covered_in_m2', 'lat', 'lon', 'borough']
target = 'price_aprox_usd'


X_train = df[features]
y_train = df[target]




In [9]:
y_mean = y_train.mean()
y_mean

54246.53149826422

In [10]:
y_pred_baseline = [y_mean] * len(y_train)
# print(y_pred_baseline[:5])


mae_baseline = mean_absolute_error(y_train, y_pred_baseline)
print(mae_baseline)

17239.939475888295


In [ ]:
# INstantiate transformer
ohe = OneHotEncoder(cols=['borough'], use_cat_names=True)
# fitting
ohe.fit(X_train)

In [ ]:
# transformation
XT_train = ohe.fit_transform(X_train)
print(XT_train.shape)
XT_train.head()

In [ ]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
#     LinearRegression()    
    Ridge()
)

model.fit(X_train, y_train)

In [11]:
ohe = OneHotEncoder(cols=['borough'], use_cat_names=True)


In [13]:
# Instantiate OneHotEncoder and specify the categorical column
ohe = OneHotEncoder(cols=['borough'], use_cat_names=True)

# Fit and transform
XT_train = ohe.fit_transform(X_train)

# Check results
XT_train.head()


,surface_covered_in_m2,lat,lon,borough_Benito Juárez,borough_Iztacalco,borough_Azcapotzalco,borough_Coyoacán,borough_Álvaro Obregón,borough_Iztapalapa,borough_Cuauhtémoc,borough_Tláhuac,borough_Miguel Hidalgo,borough_Venustiano Carranza,borough_Tlalpan,borough_Gustavo A. Madero,borough_Xochimilco,borough_La Magdalena Contreras,borough_Cuajimalpa de Morelos
0,57.0,23.634501,-102.552788,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,56.0,19.402413,-99.095391,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,80.0,19.357820,-99.149406,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,60.0,19.504985,-99.208557,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,50.0,19.354219,-99.126244,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
